In [91]:
import mitsuba as mi
mi.variants()
mi.set_variant("cuda_ad_rgb")
scene = mi.load_file("lego/cbox.xml")

In [92]:
# image = mi.render(scene, spp=16)
# import matplotlib.pyplot as plt
# plt.axis("off")
# plt.imshow(image ** (1.0 / 2.2)); # approximate sRGB tonemapping

In [93]:
from mitsuba import ScalarTransform4f as T
def load_sensor(r, phi, theta):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    # origin = T.rotate([0, 0, 1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 4, r])
    origin = T.translate([0, -1.2, 0]) @ T.rotate([0, 1, 0], phi).rotate([1, 0, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.37,
        'to_world': T.look_at(
            origin=origin,
             target=[0, -3.7, 0],
            up=[0, 1, 0]
        ),
        # 'sampler': {
            # 'type': 'independent',
            # 'sample_count': 64
        # },
        # 'film': {
            # 'type': 'hdrfilm',
            # 'width': 1920,
            # 'height': 1080,
            # 'rfilter': {
                # 'type': 'tent',
            # },
            # 'pixel_format': 'rgb',
        # },
    })

In [94]:
radius = 2
phis = [20.0*(i+1) for i in range(10)]
thetas = [20.0*(j+1) for j in range(10)]
# thetas = [90]

sensors = []
for phi in phis:
        for theta in thetas:
            sensors.append(load_sensor(radius,theta,phi))
sensor_count = len(sensors)

In [95]:
# sensor_count = 90
# radius = 3
# phis = [20.0 * i for i in range(sensor_count)]
# theta = 90.0
# sensors = [load_sensor(radius, phi, theta) for phi in phis]

In [96]:
images = [mi.render(scene, spp=512, sensor=sensor) for sensor in sensors]

In [97]:
import os
folder_name = "rendered_images"

# Create the folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)

# Save the rendered images with descriptive filenames
for i, image in enumerate(images):
    filename = f"{folder_name}/view_{i+1:04d}.png"
    mi.util.write_bitmap(filename, image)
    
print(f"Rendered images saved to: {folder_name}")

Rendered images saved to: rendered_images
